In [58]:
import torch
import random
import numpy as np
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel
from setfit import SetFitTrainer
from sklearn.metrics import classification_report, confusion_matrix
from datasets import Dataset
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [59]:
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)

In [65]:
df = pd.read_csv('./data_unlabeled/all_unlabeled.csv')
labeled_df = pd.read_csv('./data_labeled/all_labeled.csv')

test_set = Dataset.from_pandas(df)
labeled_test_set = Dataset.from_pandas(labeled_df)

In [74]:
model = SetFitModel.from_pretrained("PeppoCola/FewShotIssueClassifier-NLBSE23")

/Users/jchang15/anaconda3/envs/defects/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [68]:
y_pred = model.predict(labeled_test_set['text'])

print(classification_report(labeled_test_df['label'], y_pred, digits=4))
print(confusion_matrix(labeled_test_df['label'], y_pred))

              precision    recall  f1-score   support

         0.0     0.6172    0.5308    0.5707       878
         1.0     0.4030    0.7378    0.5213       431
         2.0     0.7513    0.6228    0.6810      1678
         3.0     0.3588    0.5949    0.4476        79

    accuracy                         0.6119      3066
   macro avg     0.5326    0.6216    0.5552      3066
weighted avg     0.6538    0.6119    0.6210      3066

[[ 466  101  266   45]
 [  33  318   73    7]
 [ 233  368 1045   32]
 [  23    2    7   47]]


In [69]:
# y_pred = model.predict(test_set['text'])

# plt.hist(y_pred)

In [70]:
# bug = labeled_df.loc[labeled_df['label'] == 0]
# docs = labeled_df.loc[labeled_df['label'] == 1].sample(50)
# feature = labeled_df.loc[labeled_df['label'] == 2].sample(50)
# question = labeled_df.loc[labeled_df['label'] == 3].sample(50)

bug_train, bug_test = train_test_split(labeled_df.loc[labeled_df['label'] == 0], random_state=seed, train_size=50, test_size=50)
docs_train, docs_test = train_test_split(labeled_df.loc[labeled_df['label'] == 1], random_state=seed, train_size=50, test_size=50)
feature_train, feature_test = train_test_split(labeled_df.loc[labeled_df['label'] == 2], random_state=seed, train_size=50, test_size=50)
question_train, question_test = train_test_split(labeled_df.loc[labeled_df['label'] == 3], random_state=seed, train_size=50)

train_df = pd.concat([bug_train, docs_train, feature_train, question_train], ignore_index=True)
test_df = pd.concat([bug_test, docs_test, feature_test, question_test], ignore_index=True)

train_set = Dataset.from_pandas(train_df)
test_set = Dataset.from_pandas(test_df)

In [75]:
y_pred = model.predict(test_set['text'])

print(classification_report(test_set['label'], y_pred, digits=4))
print(confusion_matrix(test_set['label'], y_pred))

              precision    recall  f1-score   support

         0.0     0.5116    0.4400    0.4731        50
         1.0     0.6136    0.5400    0.5745        50
         2.0     0.4118    0.5600    0.4746        50
         3.0     0.6667    0.5517    0.6038        29

    accuracy                         0.5196       179
   macro avg     0.5509    0.5229    0.5315       179
weighted avg     0.5373    0.5196    0.5230       179

[[22  6 19  3]
 [ 3 27 18  2]
 [ 9 10 28  3]
 [ 9  1  3 16]]


In [76]:
model = SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_set,
    eval_dataset=test_set,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    num_epochs=1,
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [77]:
trainer.train()

Generating Training Pairs: 100%|███████████████| 20/20 [00:00<00:00, 157.13it/s]
***** Running training *****
  Num examples = 8000
  Num epochs = 1
  Total optimization steps = 500
  Total train batch size = 16
Iteration:  19%|█████▍                       | 93/500 [25:58<1:51:27, 16.43s/it]


Iteration:  38%|██████████▋                 | 190/500 [54:39<1:30:17, 17.47s/it]


Iteration:  57%|██████████████▉           | 287/500 [1:22:59<1:00:01, 16.91s/it]


Iteration:  77%|█████████████████████▌      | 384/500 [1:51:33<35:25, 18.32s/it]


Iteration:  96%|██████████████████████████▉ | 481/500 [2:20:44<05:43, 18.09s/it]


Epoch: 100%|██████████████████████████████████| 1/1 [2:26:17<00:00, 8777.17s/it]


In [78]:
metrics = trainer.evaluate()
metrics

***** Running evaluation *****


{'accuracy': 0.7150837988826816}

In [79]:
trainer.model._save_pretrained('model')

In [80]:
y_pred = model.predict(test_set['text'])

print(classification_report(test_set['label'], y_pred, digits=4))
print(confusion_matrix(test_set['label'], y_pred))

              precision    recall  f1-score   support

         0.0     0.6875    0.6600    0.6735        50
         1.0     0.8409    0.7400    0.7872        50
         2.0     0.5763    0.6800    0.6239        50
         3.0     0.8571    0.8276    0.8421        29

    accuracy                         0.7151       179
   macro avg     0.7405    0.7269    0.7317       179
weighted avg     0.7268    0.7151    0.7187       179

[[33  3 11  3]
 [ 1 37 12  0]
 [12  3 34  1]
 [ 2  1  2 24]]
